In [51]:
# Дадена е раница с вместимост M килограма и N предмета, всеки от които се характеризира с две числа - тегло mi и стойност ci. Използвайки Генетичен алгоритъм, да се избере такова множество от предмети, чиято сумарна стойност е максимална, а сумата от теглата не надвишава M. За целта нека да се изведе на пет стъпки стойността на текущо най-добрата конфигурация от предмети в популацията.
# 1. на 10-та генерация
# 2, 3, 4 - по избор
# 5. След последната генерация

# N < 10 000

# Вход:
# M, N
# N реда определящи mi и ci

# Изход:
# Максималната възможна такава сума от стойности 

In [52]:
# READING INPUT FROM FILE
import InputReader
import numpy as np
M, N, weight, cost = InputReader.read_input("./data/KP long test data.txt")

In [53]:
# Fitness for KP: if the weight sum(v_i*w_i) > M it's zero, otherwise sum(v_i*c_i)
def fitness(v):
    # INPUT: bool vector v of size N
    # OUTPUT: fitness value
    
    # dot(v,w) = sum(v_i*w_i)
    # dot(v,c) = sum(v_i*c_i)
    return 0 if np.dot(v,weight) > M else np.dot(v,cost)


In [54]:
# Generate one N-length bool vector that has fitness > 0 
def generate_some():
    ind = np.zeros(N,np.int8)
    # Random amount of included items
    k = abs(int(np.trunc(np.random.normal(loc=N/2,scale=N/4))))
    # Random indices to try and include
    add = np.random.randint(N,size=k)
    for i in add:
        ind[i] = 1 # add item
        if fitness(ind) == 0: # if it overloads the bag (fitness = 0) then remove it
            ind[i] = 0
    return ind.tolist()

# Generate one N-length bool vector that has fitness > 0 
def generate_all():
    ind = np.zeros(N,np.int8)
    # Shuffle indices and try to include each one 
    l = np.arange(N)
    np.random.shuffle(l)
    for i in l:
        ind[i] = 1 # add item
        if fitness(ind) == 0: # if it overloads the bag (fitness = 0) then remove it
            ind[i] = 0
    return ind.tolist()


In [55]:
# backpack representation: bool vector of size N
# starting population: generate a bunch of bool vectors
def starting_population(k, gen_alg="trysome"):
    # generate k vectors of size N => shape (k,N)
    if gen_alg == "random":
        return np.random.randint(2,size=(k,N)).tolist()

    pop = []
    if gen_alg == "tryall":
        while len(pop) < k:
            pop.append(generate_all())
    else:
        while len(pop) < k:
            pop.append(generate_some())
    return pop

In [56]:
# Tournament selection: select ONE INDIVIDUAL to become a parent for the next generation
import random
def selection(pop, t=2):
    # INPUT: the population, tournament size t
    # OUTPUT: a parent individual
    ops = random.sample(pop,t) # randomly choose t (without replacement)
    return max(ops,key= lambda ind: fitness(ind)) # sort them by fitness and choose the best


In [57]:
# Crossover: Choose two parents then crossover to produce offspring
def crossover(pop, t=2, alg="op"):
    # INPUT: the population, the tournament size & the type of crossover
    # op : one-point crossover
    # 2p : two-point crossover
    # unif : uniform crossover
    # OUTPUT: crossover child
    p1 = selection(pop,t)
    p2 = selection(pop,t)
    
    if alg == "op":
        # select cut-point from [1,N-1] from normal distribution N(mean=len/2,std=len/4)
        cut = int(np.trunc(np.random.normal(loc=N/2,scale=N/4)))
        return p1[:cut] + p2[cut:]
    elif alg == "2p":
         # select 2 cut-points from [1,N-1] from normal distribution N(mean=len/2,std=len/4)
        cut1, cut2 = np.trunc(np.random.normal(loc=N/2,scale=N/4,size=2))
        cut1 = abs(int(cut1))
        cut2 = abs(int(cut2))
        if cut1 > cut2:
            temp = cut1
            cut1 = cut2
            cut2 = temp
        return p1[:cut1]+p2[cut1:cut2]+p1[cut2:] # p1[0,cut1)+p2[cut1,cut2)+p1[cut2,]
    elif alg == "unif":
        # randomly select p1[i] or p2[i] for child[i]
        # if 1: pick p1 if 2: pick p2
        return [p1[index] if val == 1 else p2[index] for index, val in enumerate(np.random.randint(1,3,size=N))]
                


In [58]:
# Mutation: Choose a 'parent' to create a mutated 'offspring'
def mutation(pop,t=2):
    # INPUT: the population, the tournament size & probability for a single bit to mutate (flip)
    # OUTPUT: mutant child
    p = selection(pop,t)
    # decide how many flips to perform
    k = np.random.randint((1/3)*N,(2/3)*N)
    # randomly choose which to flip
    indices = random.sample(p,k)

    def flip(i):
        return 0 if i == 1 else 0
        
    #flip them
    child = p.copy()
    for i in indices:
        flip(child[i])
    return child
def mutation_2(pop, t=2):
        # INPUT: the population, the tournament size & probability for a single bit to mutate (flip)
    # OUTPUT: mutant child
    def flip(i):
        return 0 if i == 1 else 0
    
    p = selection(pop,t)
    child = p.copy()

    num = 20 
    for _ in range(num):
        # decide how many flips to perform
        k = np.random.randint((1/3)*N,(2/3)*N)
        # randomly choose which to flip
        indices = random.sample(p,k)  
        #flip them
        for i in indices:
            flip(child[i])
    
    return child


In [59]:
# Elitism: carry over the best k individuals over to the next generation
def elitism(pop,k=10):
    return sorted(pop,key=lambda ind:fitness(ind),reverse=True)[:k]


In [60]:
from collections import deque
import time

def solve_KP(mut="normal", k=100, gen_alg = "trysome", elite_prop=0.1,xo_prop=0.85, t = 2, xo_alg = "op", method = "epochs", epochs=1000, max_gens=30, alpha=0.02, s=60,quiet=True):
    # Create next generation
    def iteration(gen):
        # INPUT: generation
        # OUTPUT: new generation
        new = []
        elites = elitism(gen, int(elite_prop*k)) # retain elite_prop% of elites
        for _ in range(int(xo_prop*k)): # create xo_prop% crossover children
            new.append(crossover(gen, t, xo_alg))
        for _ in range(int((1-elite_prop-xo_prop)*k)):
            if mut == "20":
                new.append(mutation_2(gen,t))
            else:
                new.append(mutation(gen,t))
        return new + elites

    # Create starting population of size k
    gen = starting_population(k, gen_alg)

    if method == "epochs": # stops after epochs iterations
        for i in range(epochs):
            if not quiet:
                if i in [10,int(epochs/4), int(epochs/2), int((3*epochs)/4)]:
                    print("Best fitness score of generation", i, ":", np.max([fitness(i) for i in gen]))
            gen = iteration(gen)
    elif method == "stall": # stops when the average relative change in the fitness function value over max_gens is less than alpha
        def avg_fitness(pop):
            return np.average([fitness(ind) for ind in pop])
        
        def relative_change(hist):
            changes = []
            copy = hist.copy()
            copy.popleft()
            for old, new in zip(hist,copy):
                change = (avg_fitness(new) - avg_fitness(old)) / avg_fitness(old)
                changes.append(change)
            return np.average(changes)

        # deque of history 
        history = deque(maxlen=max_gens)
        history.append(gen)

        # generation counter
        i = 0
        while len(history) < max_gens or relative_change(history) < alpha:
            if not quiet:
                if i in [10, 20, 30]:
                    print("Best fitness score of generation", i, ":", np.max([fitness(i) for i in gen]))
            gen = iteration(gen)
            history.append(gen)
            i = i+1
    else:  # stops when max time (s) was exceeded
        t_end = time.time() + s
        # generation counter
        i = 0
        while time.time() < t_end:
            if not quiet:
                if i in [10,20,30]:
                    print("Best fitness score of generation", i, ":", np.max([fitness(i) for i in gen]))
            gen = iteration(gen)
            i=i+1

    sol = max(gen,key=lambda i: fitness(i))
    if not quiet:
        print("Best fitness score of last generation:", fitness(sol))
    return sol, fitness(sol)


In [64]:
one_mutation = []
for i in range(5):
    _, fit_score = solve_KP(method="stall",xo_alg="unif",k=300,alpha=0.02,max_gens=50,quiet=False)
    one_mutation.append(fit_score)

Best fitness score of generation 10 : 4455
Best fitness score of generation 20 : 5072
Best fitness score of generation 30 : 5072
Best fitness score of last generation: 5072
Best fitness score of generation 10 : 4546
Best fitness score of generation 20 : 4985
Best fitness score of generation 30 : 5053
Best fitness score of last generation: 5053
Best fitness score of generation 10 : 4347
Best fitness score of generation 20 : 4953
Best fitness score of generation 30 : 5044
Best fitness score of last generation: 5072
Best fitness score of generation 10 : 4333
Best fitness score of generation 20 : 4892
Best fitness score of generation 30 : 4984
Best fitness score of last generation: 4984
Best fitness score of generation 10 : 4199
Best fitness score of generation 20 : 4902
Best fitness score of generation 30 : 5053
Best fitness score of last generation: 5053


In [65]:
twenty_mutation = []
for i in range(5):
    _, fit_score = solve_KP(mut="20", method="stall",xo_alg="unif",k=300,alpha=0.02,max_gens=50,quiet=False)
    twenty_mutation.append(fit_score)

Best fitness score of generation 10 : 4225
Best fitness score of generation 20 : 5119
Best fitness score of generation 30 : 5119
Best fitness score of last generation: 5119
Best fitness score of generation 10 : 4143
Best fitness score of generation 20 : 5119
Best fitness score of generation 30 : 5119
Best fitness score of last generation: 5119
Best fitness score of generation 10 : 4539
Best fitness score of generation 20 : 5044
Best fitness score of generation 30 : 5101
Best fitness score of last generation: 5101
Best fitness score of generation 10 : 4233
Best fitness score of generation 20 : 5028
Best fitness score of generation 30 : 5119
Best fitness score of last generation: 5119
Best fitness score of generation 10 : 4624
Best fitness score of generation 20 : 5072
Best fitness score of generation 30 : 5072
Best fitness score of last generation: 5072


In [66]:
print(one_mutation)
print(twenty_mutation)

[5072, 5053, 5072, 4984, 5053]
[5119, 5119, 5101, 5119, 5072]


In [67]:
print(np.average(one_mutation))
print(np.average(twenty_mutation))

5046.8
5106.0
